# Weber-Fechner-Gesetz

Das **Weber-Fechner-Gesetz** beschreibt den Zusammenhang zwischen **physikalischer Reizstärke** $R$
und der **empfundenen Intensität** $E$.

### Weber’sches Gesetz

$$
\frac{\Delta R}{R} = k
$$

- $R$: Reizstärke  
- $\Delta R$: ebenmerkliche Reizänderung  
- $k$: Weber-Konstante (abhängig von der Sinnesmodalität)

Das bedeutet: Die kleinste wahrnehmbare Änderung ist **proportional** zum Ausgangsreiz.

 

### Fechner’sches Gesetz

Fechner kombinierte Webers Befund mit einer logarithmischen Integration:

$$
E = k \cdot \ln\left(\frac{R}{R_0}\right)
$$

- $R_0$: Reizschwelle  
- $k$: Proportionalitätsfaktor  
- $E$: Empfindungsintensität  

→ Die Wahrnehmung wächst **logarithmisch** mit dem physikalischen Reiz.

 

## Anwendung in der Akustik

### Schalldruckpegel

In der Akustik beschreibt der **Schalldruckpegel** $L_p$ die Intensität des Schalls:

$$
L_p = 20 \cdot \log_{10}\left(\frac{p}{p_0}\right)
$$

- $p$: Effektivwert des Schalldrucks  
- $p_0 = 20 \, \mu \text{Pa}$: Hörschwelle bei 1 kHz  

Die **logarithmische Skala** der Dezibel ist also direkt vom Fechner’schen Ansatz abgeleitet.

 

### Wahrnehmung der Lautstärke

Die **Lautheit** wird nicht linear zur physikalischen Intensität empfunden:

- +10 dB → ungefähr **Verdopplung der empfundenen Lautstärke**
- Beispiel:  
  - Sprache: 60 dB  
  - Staubsauger: 70 dB  
  - → klingt etwa doppelt so laut, obwohl die Schallintensität **10× größer** ist.

 

### Weber’sches Gesetz in der Akustik: Pegeländerung und Frequenzänderung

Die kleinste wahrnehmbare **Pegeländerung** (JND: just noticeable difference) beträgt etwa:

$$
\Delta L \approx 1\,\text{dB}
$$

Dies bleibt über weite Pegelbereiche **konstant**, also gilt:

$$
\frac{\Delta L}{L} = \text{konstant}
$$


Auch die empfundene Frequenzänderung ist Abhängig vom Verhältnis der Frequenzänderung.

In den untenstehenden Zellen sind Code Beispiele, um die Pegeländerungen und Frequenzänderungen darzustellen.

# Pegeländerung

Im folgenden Code wird die Änderung der Pegel verdeutlicht.

Eine Änderung von 10dB ist gefühlt doppelt so laut. Um weiter 10dB ist auch das wieder eine Verdopplung der wahrgenommen Lautstärke.

Ein Pegelunterschied von 1dB kann kaum wahrgenommen werden.

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import Audio, display, HTML
from matplotlib.gridspec import GridSpec

FS = 44100
color_signal1 = "#00FF00"
color_signal2 = "#FFFF00"
color_signal3 = "#00FFFF"

# --- Signal-Generator ---
def generate_signals(frequency, duration, amplitude, dB2, dB3):
    t = np.linspace(0, duration, int(FS * duration), endpoint=False)
    s1 = amplitude * np.sin(2 * np.pi * frequency * t)
    
    A2 = amplitude * 10 ** (dB2 / 20)
    s2 = np.clip(A2 * np.sin(2 * np.pi * frequency * t), -1.0, 1.0)
    s2_no_clip = A2 * np.sin(2 * np.pi * frequency * t)
    
    A3 = amplitude * 10 ** (dB3 / 20)
    s3 = np.clip(A3 * np.sin(2 * np.pi * frequency * t), -1.0, 1.0)
    s3_no_clip = A3 * np.sin(2 * np.pi * frequency * t)
    
    return (t.astype(np.float32), s1.astype(np.float32), s2.astype(np.float32),
            s3.astype(np.float32), s2_no_clip.astype(np.float32), s3_no_clip.astype(np.float32))

# --- Output-Widgets ---
plot_out = widgets.Output(layout=widgets.Layout(border='1px solid #333', padding='6px'))
audio_out = widgets.Output(layout=widgets.Layout(border='1px solid #333', padding='6px'))

# --- Plot-Funktion ---
def plot_signals(frequency, duration, amplitude, dB2, dB3):
    t, s1, s2, s3, s2_no_clip, s3_no_clip = generate_signals(frequency, duration, amplitude, dB2, dB3)
    with plot_out:
        plot_out.clear_output(wait=True)
        plt.style.use("dark_background")

        fig = plt.figure(figsize=(10, 5))
        gs = GridSpec(2, 3, height_ratios=[1.2, 1])
        ax_time = fig.add_subplot(gs[0, :])
        ax_fft1 = fig.add_subplot(gs[1, 0])
        ax_fft2 = fig.add_subplot(gs[1, 1])
        ax_fft3 = fig.add_subplot(gs[1, 2])

        # --- Zeitplot ---
        ax_time.plot(t[:1000]*1000, s1[:1000], color=color_signal1, label="Signal 1")
        ax_time.plot(t[:1000]*1000, s2[:1000], color=color_signal2, label="Signal 2")
        ax_time.plot(t[:1000]*1000, s3[:1000], color=color_signal3, label="Signal 3")
        ax_time.set_xlim(0, 20)
        ax_time.set_ylim(-1,1)
        ax_time.set_xlabel("Zeit (ms)")
        ax_time.set_ylabel("Amplitude")
        ax_time.legend(loc="upper right")
        ax_time.grid(True, color="#444")

        # --- FFT-Funktion ---
        def fft_plot(sig, ax, color, label_y=False):
            N = len(sig)
            fft_vals = np.fft.rfft(sig)
            fft_freqs = np.fft.rfftfreq(N, 1/FS)
            mag = 20 * np.log10(np.abs(fft_vals) + 1e-12) + 20*np.log10(1/(FS*duration/2))
            ax.semilogx(fft_freqs, mag, color=color)
            ax.set_xlim(100, 20000)
            ax.set_ylim(-70, 0)
            if label_y:
                ax.set_ylabel("Amplitude (dBFS)")
            ax.set_xlabel("Frequenz (Hz)")
            ax.grid(True, which="both", ls="--", color="#444")

        fft_plot(s1, ax_fft1, color_signal1, True)
        fft_plot(s2, ax_fft2, color_signal2)
        fft_plot(s3, ax_fft3, color_signal3)

        plt.subplots_adjust(hspace=0.35, bottom=0.1, top=0.95, left=0.08, right=0.98)
        plt.show()

        # --- Info-Labels ---
        display(HTML(f"<b>Signal 1: Peak={np.max(np.abs(s1)):.3f}</b>"))
        display(HTML(f"<b>Signal 2: Peak={np.max(np.abs(s2_no_clip)):.3f} | A2/A1={np.max(np.abs(s2_no_clip))/np.max(np.abs(s1)):.2f}</b>"))
        display(HTML(f"<b>Signal 3: Peak={np.max(np.abs(s3_no_clip)):.3f} | A3/A1={np.max(np.abs(s3_no_clip))/np.max(np.abs(s1)):.2f}</b>"))

# --- Audio-Funktionen ---
def play_single(frequency, duration, amplitude, dB2, dB3, idx=1, autoplay=False):
    plot_signals(frequency, duration, amplitude, dB2, dB3)
    _, s1, s2, s3, s2_no_clip, s3_no_clip = generate_signals(frequency, duration, amplitude, dB2, dB3)
    sigs = [s1, s2, s3]
    sig = sigs[idx-1]
    with audio_out:
        audio_out.clear_output(wait=True)
        if idx == 1:
            display(HTML(f"<b>Signal 1: Peak={np.max(np.abs(s1)):.3f}</b>"))
        elif idx == 2:
            display(HTML(f"<b>Signal 2: Peak={np.max(np.abs(s2_no_clip)):.3f} | Amplitudenverhältnis A2/A1={np.max(np.abs(s2_no_clip))/np.max(np.abs(s1)):.2f}</b> (des nicht-geclippten Signals)"))
        elif idx == 3:
            display(HTML(f"<b>Signal 3: Peak={np.max(np.abs(s3_no_clip)):.3f} | Amplitudenverhältnis A3/A1={np.max(np.abs(s3_no_clip))/np.max(np.abs(s1)):.2f}</b> (des nicht-geclippten Signals)"))
        display(Audio(data=sig, rate=FS, autoplay=autoplay, normalize=False))

def play_all(frequency, duration, amplitude, dB2, dB3, gap_s=0.05, autoplay=False):
    plot_signals(frequency, duration, amplitude, dB2, dB3)
    _, s1, s2, s3, _, _ = generate_signals(frequency, duration, amplitude, dB2, dB3)
    gap = np.zeros(int(FS*gap_s), dtype=np.float32)
    s_all = np.concatenate([s1, gap, s2, gap, s3])
    with audio_out:
        audio_out.clear_output(wait=True)
        display(HTML(f"<b>Alle Signale</b>"))
        display(Audio(data=s_all, rate=FS, autoplay=autoplay, normalize=False))

# --- Widgets ---
frequency_slider = widgets.FloatSlider(value=1000.0, min=50.0, max=15000.0, step=1.0, description='Freq (Hz)', layout=widgets.Layout(width='420px'))
duration_slider = widgets.FloatSlider(value=1.25, min=0.05, max=5.0, step=0.05, description='Dauer (s)', layout=widgets.Layout(width='420px'))
amplitude_slider = widgets.FloatSlider(value=0.1, min=0.01, max=1.0, step=0.01, description='Amplitude S1', layout=widgets.Layout(width='420px'))
db2_slider = widgets.FloatSlider(value=10.0, min=-60.0, max=60.0, step=0.01, description='ΔdB S1 -> S2', layout=widgets.Layout(width='420px'))
db3_slider = widgets.FloatSlider(value=20.0, min=-60.0, max=60.0, step=0.01, description='ΔdB S1 -> S3', layout=widgets.Layout(width='420px'))

btn_plot = widgets.Button(description="Plot", button_style='info')
btn_play1 = widgets.Button(description="Play S1", style={'button_color': color_signal1})
btn_play2 = widgets.Button(description="Play S2", style={'button_color': color_signal2})
btn_play3 = widgets.Button(description="Play S3", style={'button_color': color_signal3})
btn_play_all = widgets.Button(description="Play All", button_style='danger')
chk_autoplay = widgets.Checkbox(value=True, description='Autoplay')

# --- Callbacks ---
btn_plot.on_click(lambda b: plot_signals(frequency_slider.value, duration_slider.value, amplitude_slider.value, db2_slider.value, db3_slider.value))
btn_play1.on_click(lambda b: play_single(frequency_slider.value, duration_slider.value, amplitude_slider.value, db2_slider.value, db3_slider.value, idx=1, autoplay=chk_autoplay.value))
btn_play2.on_click(lambda b: play_single(frequency_slider.value, duration_slider.value, amplitude_slider.value, db2_slider.value, db3_slider.value, idx=2, autoplay=chk_autoplay.value))
btn_play3.on_click(lambda b: play_single(frequency_slider.value, duration_slider.value, amplitude_slider.value, db2_slider.value, db3_slider.value, idx=3, autoplay=chk_autoplay.value))
btn_play_all.on_click(lambda b: play_all(frequency_slider.value, duration_slider.value, amplitude_slider.value, db2_slider.value, db3_slider.value, gap_s=0.05, autoplay=chk_autoplay.value))

controls = widgets.VBox([
    widgets.HBox([amplitude_slider, frequency_slider, duration_slider]),
    widgets.HBox([db2_slider]),
    widgets.HBox([db3_slider]),
    widgets.HBox([btn_play1, btn_play2, btn_play3, btn_play_all, chk_autoplay]) #btn_plot
])

display(controls)
display(audio_out)
display(plot_out)

# --- Automatisches Update bei Änderung eines Sliders ---
def update_plot(change):
    plot_signals(
        frequency_slider.value,
        duration_slider.value,
        amplitude_slider.value,
        db2_slider.value,
        db3_slider.value
    )

# Alle Slider beobachten
for slider in [frequency_slider, duration_slider, amplitude_slider, db2_slider, db3_slider]:
    slider.observe(update_plot, names='value')


Output(layout=Layout(border_bottom='1px solid #333', border_left='1px solid #333', border_right='1px solid #33…

Output(layout=Layout(border_bottom='1px solid #333', border_left='1px solid #333', border_right='1px solid #33…

# Frequenzänderung in Sprüngen
In diesem Code kann man die Änderung der Frequenz abhängig vom Frequenzverhältnis oder einem fixen Frequenzschritt (z.B. 50 Hz) testen.

Die Frequenzschritte klingen gleich, wenn sie über den Skalierungsfaktor gewählt werden.

Bei einer festern Schrittweite klingt die Frequenzänderung komplett unterschiedlich.

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import Audio, display, HTML

FS = 44100
AMPLITUDE = 0.33
pair_colors = ['#00FF00', '#FFFF00', '#00FFFF']  # Farben für 3 Paare

# --- Hilfsfunktionen ---
def safe_clip(signal):
    signal = np.nan_to_num(signal, nan=0.0, posinf=1.0, neginf=-1.0)
    return np.clip(signal, -1.0, 1.0).astype(np.float32)

def generate_scaled_signals(duration, freqs, scale_factor, use_fixed_diff=False, freq_diff=0.0):
    """Erzeugt drei Signalpaare (Basis und skaliert) mit wahlweise Skalierungsfaktor oder Frequenzdifferenz."""
    t = np.linspace(0, duration, int(FS*duration), endpoint=False)
    wave_pairs = []
    for f0 in freqs:
        f_scaled = f0 * scale_factor if not use_fixed_diff else f0 + freq_diff
        w_base = safe_clip(AMPLITUDE * np.sin(2*np.pi*f0*t))
        w_scaled = safe_clip(AMPLITUDE * np.sin(2*np.pi*f_scaled*t))
        wave_pairs.append((w_base, w_scaled, f_scaled))
    return t, wave_pairs

# --- Plot: nur FFT ---
plot_out = widgets.Output(layout=widgets.Layout(border='1px solid #333', padding='6px'))
audio_out = widgets.Output(layout=widgets.Layout(border='1px solid #333', padding='6px'))

def plot_ffts(duration, freqs, scale_factor, use_fixed_diff, freq_diff):
    t, wave_pairs = generate_scaled_signals(duration, freqs, scale_factor, use_fixed_diff, freq_diff)
    with plot_out:
        plot_out.clear_output(wait=True)
        plt.style.use("dark_background")
        fig, ax_fft = plt.subplots(figsize=(10,5))

        N = len(t)
        fft_freqs = np.fft.rfftfreq(N, 1/FS)

        for i, (w_base, w_scaled, f_scaled) in enumerate(wave_pairs):
            fft_vals_base = np.fft.rfft(w_base)
            fft_vals_scaled = np.fft.rfft(w_scaled)
            mag_base = 20*np.log10(np.abs(fft_vals_base)+1e-12) + 20*np.log10(1/(FS*duration/2))
            mag_scaled = 20*np.log10(np.abs(fft_vals_scaled)+1e-12) + 20*np.log10(1/(FS*duration/2))

            ax_fft.semilogx(fft_freqs, mag_base, color=pair_colors[i], linestyle='-', linewidth=1.5,
                            label=f"Paar {i+1} Basis ({freqs[i]:.1f} Hz)")
            ax_fft.semilogx(fft_freqs, mag_scaled, color=pair_colors[i], linestyle='--', linewidth=1.5,
                            label=f"Paar {i+1} Skaliert ({f_scaled:.1f} Hz)")

        ax_fft.set_xlim(20, 20000)
        ax_fft.set_ylim(-70, 0)
        ax_fft.set_xlabel("Frequenz (Hz)")
        ax_fft.set_ylabel("Amplitude (dBFS)")
        ax_fft.grid(True, which='both', ls='--')
        ax_fft.legend(loc="upper right")
        plt.show()

# --- Audio ---
def play_signal_pair(idx, freqs, scale_factor, duration, use_fixed_diff=False, freq_diff=0.0, autoplay=True):
    """Spielt nacheinander: Basis, Pause, skaliert."""
    t, wave_pairs = generate_scaled_signals(duration, freqs, scale_factor, use_fixed_diff, freq_diff)
    base, scaled, f_scaled = wave_pairs[idx]

    pause = np.zeros(int(0.2*FS), dtype=np.float32)  # 200 ms Pause
    combined = np.concatenate([base, pause, scaled])

    with audio_out:
        audio_out.clear_output(wait=True)
        display(HTML(
            f"<b>Paar {idx+1}: {freqs[idx]:.1f} Hz → {f_scaled:.1f} Hz</b>"
        ))
        display(Audio(data=combined, rate=FS, autoplay=autoplay, normalize=False))

# --- Widgets ---
duration_slider = widgets.FloatSlider(value=1.0, min=0.2, max=5.0, step=0.05,
                                      description='Dauer (s)', layout=widgets.Layout(width='420px'))

# Frequenz-Slider (logarithmisch)
# min/max sind Exponenten: 10^1.3 ≈ 20 Hz, 10^4 = 10000 Hz
freq1_slider = widgets.FloatLogSlider(value=100.0, min=1.3, max=4, step=0.001, base=10,
                                       description='Freq 1 (Hz)', layout=widgets.Layout(width='420px'), 
                                       readout_format='.0f')
freq2_slider = widgets.FloatLogSlider(value=200.0, min=1.3, max=4, step=0.001, base=10,
                                       description='Freq 2 (Hz)', layout=widgets.Layout(width='420px'),
                                       readout_format='.0f')
freq3_slider = widgets.FloatLogSlider(value=1000.0, min=1.3, max=4, step=0.001, base=10,
                                       description='Freq 3 (Hz)', layout=widgets.Layout(width='420px'),
                                       readout_format='.0f')

# Skalierungs- & Differenz-Slider
scale_slider = widgets.FloatSlider(value=1.25, min=1.1, max=2.0, step=0.01,
                                   description='Skalierung', layout=widgets.Layout(width='420px'))
diff_slider = widgets.FloatSlider(value=50.0, min=1.0, max=1000.0, step=1.0,
                                  description='Δf (Hz)', layout=widgets.Layout(width='420px'))
diff_slider.disabled = True  # initial deaktiviert

# Toggle Checkbox
chk_fixed_diff = widgets.Checkbox(value=False, description='Fix Frequency Distance')

btn_plot = widgets.Button(description="Plot FFT", button_style='info')
btn_play1 = widgets.Button(description="Play Paar 1", style={'button_color': pair_colors[0]})
btn_play2 = widgets.Button(description="Play Paar 2", style={'button_color': pair_colors[1]})
btn_play3 = widgets.Button(description="Play Paar 3", style={'button_color': pair_colors[2]})
chk_autoplay = widgets.Checkbox(value=True, description='Autoplay')

# --- Callbacks ---
def get_freqs():
    return [float(freq1_slider.value), float(freq2_slider.value), float(freq3_slider.value)]

def update_mode(change=None):
    """Aktiviere/Deaktiviere Slider je nach Modus"""
    if chk_fixed_diff.value:
        scale_slider.disabled = True
        diff_slider.disabled = False
    else:
        scale_slider.disabled = False
        diff_slider.disabled = True
    update_plot()

def update_plot(_=None):
    plot_ffts(duration_slider.value, get_freqs(), scale_slider.value,
              chk_fixed_diff.value, diff_slider.value)

# Button callbacks
btn_plot.on_click(lambda b: update_plot())
btn_play1.on_click(lambda b: play_signal_pair(0, get_freqs(), scale_slider.value, duration_slider.value,
                                              chk_fixed_diff.value, diff_slider.value, autoplay=chk_autoplay.value))
btn_play2.on_click(lambda b: play_signal_pair(1, get_freqs(), scale_slider.value, duration_slider.value,
                                              chk_fixed_diff.value, diff_slider.value, autoplay=chk_autoplay.value))
btn_play3.on_click(lambda b: play_signal_pair(2, get_freqs(), scale_slider.value, duration_slider.value,
                                              chk_fixed_diff.value, diff_slider.value, autoplay=chk_autoplay.value))

# Beobachter für Live-Update
for s in [freq1_slider, freq2_slider, freq3_slider, scale_slider, diff_slider, duration_slider]:
    s.observe(update_plot, names='value')
chk_fixed_diff.observe(update_mode, names='value')

# --- Layout ---
controls = widgets.VBox([
    duration_slider,
    freq1_slider,
    freq2_slider,
    freq3_slider,
    widgets.HBox([scale_slider, chk_fixed_diff, diff_slider]),
    widgets.HBox([btn_play1, btn_play2, btn_play3, chk_autoplay])
])

display(controls)
display(audio_out)
display(plot_out)

# Initial
update_plot()

Output(layout=Layout(border_bottom='1px solid #333', border_left='1px solid #333', border_right='1px solid #33…

Output(layout=Layout(border_bottom='1px solid #333', border_left='1px solid #333', border_right='1px solid #33…

# Frequenzänderung kontinuierlich

Im folgenden Code wird ein Sweep über eine gewählte Zeit abgespielt. Die Frequenz ändert sich dabei um den gewählten Faktor.

Die Änderung von 1% ist kaum zu erkennen. Ab 3% ist die Änderung schon merkbar.

In [5]:
import numpy as np
import ipywidgets as widgets
from IPython.display import Audio, display, HTML

FS = 44100
AMPLITUDE = 0.33

# --- Hilfsfunktion ---
def safe_clip(signal):
    signal = np.nan_to_num(signal, nan=0.0, posinf=1.0, neginf=-1.0)
    return np.clip(signal, -1.0, 1.0).astype(np.float32)

# --- Logarithmischer Sweep ---
def generate_chirp_log(duration, f_start, percent_increase):
    """
    Logarithmischer Sweep von f_start bis f_start*(1+percent_increase/100)
    """
    t = np.linspace(0, duration, int(FS*duration), endpoint=False)
    f_end = f_start * (1 + percent_increase/100)
    beta = np.log(f_end / f_start) / duration
    phase = 2 * np.pi * f_start * (np.exp(beta * t) - 1) / beta
    signal = AMPLITUDE * np.sin(phase)
    return safe_clip(signal), f_start, f_end

# --- Audio Output ---
audio_out = widgets.Output(layout=widgets.Layout(border='1px solid #333', padding='6px'))

def play_log_sweep(duration, f_start, percent_increase, autoplay=True):
    signal, f0, f1 = generate_chirp_log(duration, f_start, percent_increase)
    with audio_out:
        audio_out.clear_output(wait=True)
        display(HTML(f"<b>Logarithmischer Sweep:</b> {f0:.1f} Hz → {f1:.1f} Hz (+{percent_increase:.1f}%)"))
        display(Audio(data=signal, rate=FS, autoplay=autoplay, normalize=False))

# --- Widgets ---
duration_slider = widgets.FloatSlider(value=5.0, min=1.0, max=10.0, step=0.1,
                                      description='Dauer (s)', layout=widgets.Layout(width='420px'))
fstart_slider = widgets.FloatLogSlider(value=440.0, min=1.3, max=4.0, step=0.0001, base=10,
                                    description='Startfreq (Hz)', layout=widgets.Layout(width='420px'), readout_format='.0f')
percent_slider = widgets.IntSlider(value=5, min=1, max=20, step=1,
                                   description='Anstieg (%)', layout=widgets.Layout(width='420px'))
chk_autoplay = widgets.Checkbox(value=True, description='Autoplay')
btn_play_log = widgets.Button(description="Play Sweep", style={'button_color': '#FFAA00'})

# --- Callback ---
btn_play_log.on_click(lambda b: play_log_sweep(duration_slider.value, fstart_slider.value, percent_slider.value, autoplay=chk_autoplay.value))

# --- Layout ---
controls = widgets.VBox([duration_slider, fstart_slider, percent_slider, widgets.HBox([btn_play_log, chk_autoplay])])
display(controls)
display(audio_out)


Output(layout=Layout(border_bottom='1px solid #333', border_left='1px solid #333', border_right='1px solid #33…